In [1]:
# ------------------------------- IMPORT MODULES & SETUP ------------------------------------------------
# Standard Libraries
import os
# os.environ['OMP_NUM_THREADS'] = '48'
import math
import json
import datetime
import numpy as np
import time
# import nvtx

import argparse

# ------------------------------- CUSTOM FUNCTIONS ------------------------------------------------
# Custom Library
import sys
sys.path.append('../')

from proxy_apps.utils import file_reader, path_handler
from proxy_apps import ProxyTSPRD
# from proxy_apps.apps.timeseries_prediction import deepDMD, proxyDeepDMD, proxyDeepDMDMGPU, proxyDeepDMDPyTorch, proxyDeepDMDPyTorchJIT, hyperparameters

# from proxy_apps.utils.tf import TimingCallback
# from proxy_apps.utils.data.main import NpEncoder
# from proxy_apps.utils import file_reader, path_handler
# from proxy_apps.utils.data.grid import GridNetworkDataHandler, GridNetworkTFDataHandler, GridNetworkNewGen, GridDataGenPyTorch

In [2]:
## Initialize ProxyTSPRD
_REF_DIR = os.path.dirname('./')
config = file_reader.read_config('config_resnet.json')   

In [3]:
# app info
_APP_NAME = config['info']['app_name']
_FRAMEWORK = config['info']['framework']

_MIXED_PRECISION = 0

# initialize the interface
interface = ProxyTSPRD(_APP_NAME, _FRAMEWORK, _REF_DIR, _MIXED_PRECISION)
# interface = ProxyTSPRD('config.json', _MODEL_NAME, _N_EPOCHS, _BATCH_SIZE, _MIXED_PRECISION, 
#                  _MACHINE_NAME, _N_GPUS, _N_CPUS,
#                  _DATA_DIR, _MODEL_DIR, _FRAMEWORK)

In [4]:
# load data
training_data_dict = interface.load_data(config['data_params'])
training_data_dict

Training Data Directory: C:\Users\jain432\OneDrive - PNNL\Milan\Workspace\PNNL Projects\PACER\data\ImageNet\numpy
Validation Data Directory: C:\Users\jain432\OneDrive - PNNL\Milan\Workspace\PNNL Projects\PACER\data\ImageNet\numpy


{'training_data': <ParallelInterleaveDataset shapes: ((32, 32, 3), ()), types: (tf.int64, tf.int64)>,
 'val_data': <ParallelInterleaveDataset shapes: ((32, 32, 3), ()), types: (tf.int64, tf.int64)>,
 'data_type': 'int64',
 'training_data_format': 'image_data_generator'}

In [5]:
count = 0
for a in training_data_dict['val_data'].take(2):
    count += 1
#     print(a)
print(count)

2


In [6]:
# model info
_N_EPOCHS = 1
_BATCH_SIZE = 32

# machine info
_MACHINE_NAME = 'A100'
_N_GPUS = 1
_N_CPUS = 16

# optimization and scaling
_MGPU_STRATEGY = "MirroredStrategy"

interface.train_model(config['model_info'], training_data_dict, _N_EPOCHS, _BATCH_SIZE, _MACHINE_NAME, _N_GPUS, _N_CPUS, _MGPU_STRATEGY)

TensorBoard Log Directory Path:  C:\Users\jain432\OneDrive - PNNL\Milan\Workspace\PNNL Projects\PACER\logs
Suffix:  A100_ng1_nc16_e1_b32_mp0_mgpuMirroredStrategy
Model Path:  C:\Users\jain432\OneDrive - PNNL\Milan\Workspace\PNNL Projects\PACER\models\A100_ng1_nc16_e1_b32_mp0_mgpuMirroredStrategy
[INFO] Tensorflow version:  2.4.0
[INFO] Eager mode:  True
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Hyperparameters:  {'learning_rate': 0.01, 'num_epochs': 1, 'batch_size': 32, 'data_type': 'int64'}
Model Name:  ResNet50
Number of Replicas:  1
10/10 [==============================] - 36s 1s/step - loss: 11.9462 - accuracy: 3.2165e-04
INFO:tensorflow:Assets written to: C:\Users\jain432\OneDrive - PNNL\Milan\Workspace\PNNL Projects\PACER\models\A100_ng1_nc16_e1_b32_mp0_mgpuMirroredStrategy\assets
[INFO]: Time taken for model training (time module): 41.73730731010437 seconds
[INFO]: Time taken for model training (Keras): 35.8607955 secon

In [7]:
# # model info
# _N_EPOCHS = 1
# _BATCH_SIZE = 32

# # machine info
# _MACHINE_NAME = 'A100'
# _N_GPUS = 4
# _N_CPUS = 16

# # optimization and scaling
# _MGPU_STRATEGY = "MirroredStrategy"

# interface.train_model(config['model_info'], training_data_dict, _N_EPOCHS, _BATCH_SIZE, _MACHINE_NAME, _N_GPUS, _N_CPUS, _MGPU_STRATEGY)